**Welcome to the eDNAExplorer metadata extractor**

This script, which is designed to run within your own Google Drive account, does the following:
1. Imports metadata associated with your environmental DNA (eDNA) samples.  This metadata, which needs to be formatted as a csv file, needs to contain information on the location and time of the samples, as well as the spatial uncertainty associated with the location.  [Here's an example of an input metadata file](https://drive.google.com/file/d/1eEwXMww7vYn6ia-gu3KIH1nf_YTgA6Es/view?usp=sharing).  Note that it contains a number of variables beyond those required.
2. Extracts a variety of environmental and social variables at the eDNA sampling locations.  The spatial extent of this extraction is defined by the median spatial uncertainty of the sample set, the temporal range is defined as +/- 6 months of the earliest and latest sampling date.
3. Exports extracted values to a csv file in the top level directory of your Google Drive account.  [Here's an example output file](https://drive.google.com/file/d/1Ld9v0VSzyWydbU4s0aYMK_qKwRVb1vK2/view?usp=sharing) generated using the required columns from the example metadata input file.

---

To run this script please upload a copy to your Google Drive and run it using [Google Colab](https://research.google.com/colaboratory/).

In [ ]:
import ee
import folium
import functools
!pip install global-land-mask
from global_land_mask import globe
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

**Initialize script with Google Earth Engine**
In order to run this script the user will need to authenticate it's connection with Google Earth Engine.  In running the following step the user will be prompted to authenticate this script, which will involve following the link it generates and producing an API key to paste into the field generated by this step.

In [ ]:
ee.Authenticate()
ee.Initialize()

**Load sample metadata**

To start out, you will need a CSV file with one row for each sample. It must have the following columns: sample_id, latitude, longitude, sample_date, spatial_uncertainty.
The format for these columns need to be as follows: sample_id (string labeling your environmental DNA sample), latitude (degrees, with north being positive), longitude (degrees with east being positive), sample_date (yyyy-mm-dd), spatial_uncertainty (meters).

---

Please note that you will need to specify both the path to your metadata input file in Google Drive, as well as the name of the metadata input file.

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Get current working directory on your Google Drive.
CurrentDirectory = os.getcwd()

# Specify the directory on Google Drive that you want to use for accessing your metadata.
ProjectDirectory = "/drive/MyDrive/Colab Notebooks/"

# Specify metadata filename.
MetadataInput = "eDNAMetadataInputTest.csv"

#Specify full metadata file input path
csv_path = CurrentDirectory+ProjectDirectory+MetadataInput

# Read the relevant columns from the CSV into a dataframe
samples = pd.read_csv(csv_path, usecols=['sample_id', 'longitude', 'latitude', 'sample_date', 'spatial_uncertainty'])
samples = samples.astype({'sample_id': 'object', 'longitude': 'float64', 'latitude': 'float64', 'sample_date': 'object', 'spatial_uncertainty': 'float64'})

# Remove entries with missing data
samples = samples.dropna()

#Read in sample date as a date.
samples['sample_date']= pd.to_datetime(samples['sample_date'])

#Determine if samples are terrestrial or marine
lat = samples['latitude']
lon = samples['longitude']
samples['Terrestrial'] = globe.is_land(lat, lon)
samples['Marine'] = globe.is_ocean(lat, lon)
AllSamples = samples
#Create a terrestrial sample subset.
samples = samples[(samples['Terrestrial']==True)]
samples = samples[['sample_id', 'longitude', 'latitude', 'sample_date', 'spatial_uncertainty']]

#Define earliest and latest times using sample dates with a six +/- month buffer.
EarliestTime = min(samples['sample_date']) - relativedelta(months=6)
LatestTime = max(samples['sample_date']) + relativedelta(months=6)

# Calculate a buffer from the spatial uncertainty in meters.
BufferRadius = samples['spatial_uncertainty'].median()

#Define geographic bounds using sample coordinates and a +/- 0.5 degree buffer.
EastBound = max(samples['longitude']) + 0.5
WestBound = min(samples['longitude']) - 0.5
SouthBound = min(samples['latitude']) - 0.5
NorthBound = max(samples['latitude']) + 0.5

# Display the shape of the data we read in and its first few rows
print("Data shape:", samples.shape)
print(samples.head())

# Define a Feature for each sample area within the spatial uncertainty of each sample location.
sample_areas = []
for sample in samples.itertuples():
    # Store the important data as properties of the feature
    sample_areas.append(
        ee.Feature(ee.Geometry.Point(sample.longitude, sample.latitude).buffer(BufferRadius))
        .set('name', sample.sample_id)
        .set('sample_date', sample.sample_date)
        .set('longitude', sample.longitude)
        .set('latitude', sample.latitude)
        .set('spatial_uncertainty', sample.spatial_uncertainty))

# Define a FeatureCollection containing all the sample areas
sample_areas = ee.FeatureCollection(sample_areas)

**Define dataset classes**

GIS datasets can be in either vector or raster format. In Google Earth Engine, raster data is represented by an Image or ImageCollection and vector data by a FeatureCollection. You can find datasets by searching in the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets). It's also possible to instantiate an Earth Engine object from data that you have locally if it's not available through the API.

To conceptualize the data it will be helpful to wrap these Earth Engine objects in our own classes. This way we can store information we need to use the dataset alongside it: the band or property to use and how to process and display the data.


**Raster datasets**

In [5]:
class RasterDataset:
    
    def __init__(self, snippet, band, name=None, categorical=False, map_params={}):
        """
        Represent a Google Earth Engine raster dataset and the information we need to use it.
        
        snippet: The ee.Image or ee.ImageCollection object representing the desired dataset. 
            This is the snippet of code provided on the dataset's page in the Earth Engine catalog.
            e.g.: ee.Image('path') or ee.ImageCollection('path')
        band: The identifier of the desired image band e.g. 'bio01'
        name: The column name to display for the data gathered from this dataset
        categorical: Set to True if the data is not continuous. If True, takes the mode of pixels
            in the sample area rather than the median.
        map_params: Settings for how to display the data e.g. palette, min, max. 
            See param options: https://developers.google.com/earth-engine/api_docs#ee.data.getmapid
        """
        
        # self.name exists to be a common property between RasterDatasets and VectorDatasets
        self.name = name or band  # If a different name is not set, use the band name
        self.categorical = categorical
        self.map_params = map_params    
        
        # Select just the one band of interest from the image and rename it to self.name
        if isinstance(snippet, ee.ImageCollection):  
            # If it's an image collection, mosaic it into one image
            self.data = snippet.select([band], [self.name]).mosaic()
        else:
            self.data = snippet.select([band], [self.name])
        
        self.band = self.name  
        
    def get_sample_area_data(self, sample_area: ee.Feature) -> ee.Feature:
        """
        Add a new property to the input feature storing the value of self.data in the feature area
        
        param sample_area: ee.Feature with the geometry to sample over
        returns: input feature with a new property in the form 
            {self.band: mode (categorical) or median (continuous) of self.data 
                        over the feature geometry}
        """
        
        # Get the average distance from each point in the sample area to the data image
        distance = self.distance(sample_area.geometry())
        
        geometry = ee.Algorithms.If(
            distance,  # If the distance is not None,
            ee.Algorithms.If(
                distance.eq(0),  # If the distance is 0, the data covers the sample area
                sample_area.geometry(),
                sample_area.geometry().buffer(distance), # Otherwise, we need to buffer it
            ),
            # If the distance is None, the sample area is farther away from the data than the
            # range being searched by the distance kernel.
            # Continue with the sample area geometry, the reducer will return None.
            sample_area.geometry()
        )
        result = ee.Algorithms.If(
            self.categorical,  # If the data type is categorical,
            self.mode(geometry),  # Get the most common category in the sample area
            self.median(geometry)  # Otherwise it's continuous, so get the median
        )
    
        return sample_area.set(self.band, result)
    
    def distance(self, geometry: ee.Geometry) -> ee.Number:
        """
        Return the average distance from each point in the input geometry to the self.data image
        
        - This should be 0 in most cases, meaning the data covers the sample area.
        - If the sample area is within the search radius from the self.data image 
            (here set to 1000m), the distance will be returned.
        - If the sample area is outside the search radius, None is returned.
        
        This is intended to account for edges of datasets not lining up exactly with the 
        coastline, leaving some coastal sample sites just outside the self.data image.
        
        param geometry: ee.Geometry of the sample area of interest
        returns: ee.Number (0 or distance) or None
        """
        # Make an image representing the distance from any given point to the self.data image
        # This will be 0 everywhere that the self.data image covers
        distance_image = self.data.distance(
            ee.Kernel.euclidean(1000, 'meters'),
            False  # Do not exclude masked pixels
        ).select([self.band], ['distance'])  # Rename the band to 'distance'
        
        # Get the average distance from each point in the sample area to the nearest pixel of data
        distance = ee.Number(distance_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get('distance'))
        
        return distance
    
    def mode(self, geometry: ee.Geometry) -> ee.Number:
        """
        Reduce the dataset over the sample area to get the most common value in that area.
        
        param geometry: ee.Geometry of the sample area of interest
        returns: mode of pixels within the geometry as an ee.Number
        """
        mode = self.data.reduceRegion(
            reducer=ee.Reducer.mode(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get(self.band)
        return mode
    
    def median(self, geometry: ee.Geometry) -> ee.Number:
        """
        Reduce the dataset over the sample area to get the median value in that area
        
        param geometry: ee.Geometry of the sample area of interest
        returns: median of pixels within the geometry as an ee.Number
        """
        median = self.data.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get(self.band)
        return median
        


**Vector datasets**

In [6]:
class VectorDataset:
    
    def __init__(self, snippet, property: str, name=None, map_params={}):
        """
        Represent a Google Earth Engine vector dataset and the information we need to use it.
        
        snippet: The ee.FeatureCollection object representing the dataset.
            This is the 'Earth Engine snippet' displayed on the dataset's page in the Earth 
            Engine catalog. e.g.: ee.FeatureCollection('EPA/Ecoregions/2013/L3')
        property: The key of the desired feature property
        name: The column name to display for the data gathered from this dataset.
        map_params: Settings for how to display the data e.g. palette, min, max
        """
        self.data = snippet
        self.property = property
        self.name = name or property
        self.map_params = map_params
        
    # Note: This function is an argument to map(). Arguments to map() cannot print anything
    # or call getInfo(). Doing so results in an EEException: ValueNode empty
    # source: https://gis.stackexchange.com/questions/345598/mapping-simp>le-function-to-print-date-and-time-stamp-on-google-earth-engine-pyth
    def get_sample_area_data(self, sample_area: ee.Feature) -> str:
        """
        Return the value from the dataset to assign to the sample area.
        """
        # Get a FeatureCollection storing the overlaps between the sample area and the dataset
        overlaps = self.data.filterBounds(sample_area.geometry()).map(
            lambda feature: feature.intersection(sample_area.geometry())
        )

        result = ee.Algorithms.If(
            # If there is exactly 1 overlapping dataset feature, return its value
            overlaps.size().eq(1),
            sample_area.set(self.name, overlaps.first().get(self.property)),

            ee.Algorithms.If(
                # If there are 0 overlapping dataset features, return the value of the closest one
                overlaps.size().eq(0),
                sample_area.set(self.name, 
                                self.get_nearest_feature(sample_area).get(self.property)),

                # Otherwise, there must be >1 features overlapping the sample area
                # Return the value of the one with the largest overlap
                sample_area.set(self.name, 
                                self.get_predominant_feature(overlaps).get(self.property))
            )
        )

        return result

    def get_nearest_feature(self, sample_area: ee.Feature) -> str:
        """
        To be used when the sample area doesn't overlap the dataset at all.
        
        Get the dataset feature that is nearest to the sample area, and
        return the value of its dataset.property.
        
        param sample_area: ee.Feature representing the sample area of interest
        """

        # Define a filter to get all dataset features within 10000 meters of the sample area
        spatialFilter = ee.Filter.withinDistance(
            distance=10000,
            leftField='.geo',
            rightField='.geo',
            maxError=10
        )
        # Define a join that will return only the 'best' (nearest) match
        saveBestJoin = ee.Join.saveBest(
          matchKey='closestFeature',
          measureKey='distance'
        )
        # Apply the join, using the distance filter to define match quality
        # Get the only feature in the resulting FeatureCollection
        result = ee.Feature(saveBestJoin.apply(
            ee.FeatureCollection(sample_area),
            self.data,
            spatialFilter
        ).first())

        # Return the closest dataset feature
        return ee.Feature(result.get('closestFeature'))
    
    def get_predominant_feature(self, overlaps: ee.FeatureCollection) -> str:
        """
        To be used when the sample area overlaps more than one dataset feature.
        
        Return the value of 'property' for the largest overlap.
        """
        # Add 'area' as a property to each feature. This is the area in square meters 
        # of the intersection of the ecoregion feature and the sample area.
        overlaps = overlaps.map(
            lambda feature: feature.set({'area': feature.geometry().area()}))

        # Find the maximum area among all the overlaps
        max_area = overlaps.aggregate_max('area')

        # Return the overlap with the largest area
        return ee.Feature(overlaps.filter(ee.Filter.gte('area', max_area)).first())

**Instantiate your datasets**

Instantiate your chosen Earth Engine datasets in the list below.

**BioClim data**

The [BioClim dataset](https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO) has 19 bands representing different measures of temperature and precipitation to a resolution of 30 arc seconds (1 kilometer).

Some of the bands have had their values multiplied by a factor of 10 or 100 in order to store the data in integer format. To get the actual values we need to divide each pixel in the image by that factor. We can do this with the ee.Image.divide function, which we apply to the image argument to our dataset constructors. In a similar way, any snippet which evaluates to an ee.Image or ee.ImageCollection object may be used as the input to the RasterDataset constructor. This way we can preprocess the dataset if desired, before extracting values.


In [7]:
bioclim = [
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)),
        band='bio01'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)),
        band='bio02'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio03'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(100)), 
        band='bio04'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio05'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio06'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio07'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio08'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio09'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio10'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio11'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio12'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio13'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio14'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio15'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio16'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio17'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio18'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio19'
    )
]

**Concatenate the lists of datasets**

In [8]:
# Here is an extra function for reprojecting an Image to a different scale. 
# Might be useful for something, not sure what effect this has exactly

def reproject(image: ee.Image, scale: int) -> ee.Image:
    '''
    Resample and reproject an image to a different resolution.
    
    param image: Image to reproject
    param scale: Desired pixel width in meters
    '''
    return image.resample('bilinear').reproject(  # Use bilinear interpolation method
        crs=image.projection().crs(),  # Keep the same map projection
        scale=scale  # Change the scale
    )

**Soil properties**

Soil data is available at 250 meter resolution from [OpenLandMap datasets](https://developers.google.com/earth-engine/datasets/tags/openlandmap). We are selecting the values at surface level (band b0) but other bands correspond to depths of 10, 30, 60, 100, and 200 centimeters. Three of these datasets have been scaled up (shown in the 'Scale' column in the 'Bands' tab on the dataset pages) and so we divide them to get the true values.

Since these datasets have band names which aren't self explanatory, for clarity we give them an alternate name which will be used in the output.

In [9]:
soil = [
    RasterDataset(  # Soil taxonomy great groups, a classification number from 0 to 433
        snippet=ee.Image('OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01'),
        band='grtgroup',
        categorical=True
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b0',
        name='soil pH in H20 at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b10',
        name='soil pH in H20 at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b30',
        name='soil pH in H20 at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b60',
        name='soil pH in H20 at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b100',
        name='soil pH in H20 at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b200',
        name='soil pH in H20 at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b0',
        name='soil organic carbon at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b10',
        name='soil organic carbon at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b30',
        name='soil organic carbon at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b60',
        name='soil organic carbon at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b100',
        name='soil organic carbon at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b200',
        name='soil organic carbon at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b0',
        name='soil sand at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b10',
        name='soil sand at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b30',
        name='soil sand at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b60',
        name='soil sand at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b100',
        name='soil sand at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b200',
        name='soil sand at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b0',
        name='soil clay at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b10',
        name='soil clay at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b30',
        name='soil clay at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b60',
        name='soil clay at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b100',
        name='soil clay at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b200',
        name='soil clay at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b0',
        name='soil bulk density at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b10',
        name='soil bulk density at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b30',
        name='soil bulk density at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b60',
        name='soil bulk density at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b100',
        name='soil bulk density at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b200',
        name='soil bulk density at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b0',
        name='soil texture class at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b10',
        name='soil texture class at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b30',
        name='soil texture class at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b60',
        name='soil texture class at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b100',
        name='soil texture class at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02'),
        band='b200',
        name='soil texture class at 200 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b0',
        name='Percent volume soil water content at 0 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b10',
        name='Percent volume soil water content at 10 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b30',
        name='Percent volume soil water content at 30 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b60',
        name='Percent volume soil water content at 60 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b100',
        name='Percent volume soil water content at 100 cm'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01'),
        band='b200',
        name='Percent volume soil water content at 200 cm'
    )
]

**Terrain**

Elevation data for the United States is available from the Shuttle Radar Topography Mission (https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4) at a resolution of 10 meters. Slope and aspect datasets are not available, but they are a direct function of elevation. Here we preprocess the elevation image using the ee.Terrain functions to derive slope and aspect images.

In [10]:
terrain = [
    RasterDataset(
        snippet=ee.Image('CGIAR/SRTM90_V4'),
        band='elevation'
    ),
    RasterDataset(
        snippet=ee.Terrain.slope(ee.Image('CGIAR/SRTM90_V4')),
        band='slope'
    ),
    RasterDataset(
        snippet=ee.Terrain.aspect(ee.Image('CGIAR/SRTM90_V4')),
        band='aspect'
    ),
]

**Human Influence Index**

Data for the CSP gHM: Global Human Modification is derived from the [global Human Modification dataset](https://developers.google.com/earth-engine/datasets/catalog/CSP_HM_GlobalHumanModification). Which provides a cumulative measure of human modification of terrestrial lands globally at 1 square-kilometer resolution. The gHM values range from 0.0-1.0 and are calculated by estimating the proportion of a given location (pixel) that is modified, the estimated intensity of modification associated with a given type of human modification or "stressor". 5 major anthropogenic stressors circa 2016 were mapped using 13 individual datasets:

    human settlement (population density, built-up areas)
    agriculture (cropland, livestock)
    transportation (major, minor, and two-track roads; railroads)
    mining and energy production
    electrical infrastructure (power lines, nighttime lights)

In [11]:
HII = [
    RasterDataset(
        snippet=ee.ImageCollection('CSP/HM/GlobalHumanModification'),
        band='gHM'
    ),
]

**Landsat 8**

Datasets derived from composite Landsat 8 satellite imagery, taken about every 2 weeks at a 30 meter resolution, include normalized difference vegetation index (NDVI [link text](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_32DAY_NDVI)), enhanced vegetation index (EVI [link text](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_8DAY_EVI)), normalized burn ratio thermal (NBRT [link text](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NBRT)), and [greenest pixel](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_GREENEST_TOA).

This is our first ImageCollection data. An ImageCollection represents a series of images over time. We will use the ee.ImageCollection.filterDate function to filter the series to our desired date range. The filtered collection will be mosaiced together to form a single image.

    If you are interested in data from a specific point in time, you can set the date range accordingly, but be aware that data from a single day might not cover the entire area of interest. You will need to experiment with different ranges and find a minimal range that has complete coverage.

    If the timestamp is not important to your data, you should still choose an appropriate date range, though it may be broader. If you do not specify a date range, overlapping images will be composited by their order in the collection, which could give you a mix of old and new data.

In [12]:
landsat = [
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(
            EarliestTime, LatestTime
        ),
        band='NDVI',
        map_params={'min': 0, 'max': 1, 'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]}
    ),
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_EVI').filterDate(
            EarliestTime, LatestTime
        ),
        band='EVI'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NBRT').filterDate(
            EarliestTime, LatestTime
        ),
        band='NBRT'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_ANNUAL_GREENEST_TOA').filterDate(
            EarliestTime, LatestTime
        ),
        band='greenness'
    )
]

**Sentinel 2**

The [Sentinel 2 satellites](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2) continuously take multi-spectral imagery of the Earth, covering a given location once every 5 days on average since 2015. Resolution is 10, 20, or 60 meters depending on the band.

This data needs a few preprocessing steps. Because this dataset is huge with tens of thousands of images, using all of them would be very slow. We filter to a date range and filter out images with more than 20% cloudy pixels. We define a method that filters the set to only include images covering California. We also define a cloud masking function, which makes pixels transparent if the QA60 band indicates they have cloud or cirrus cover. And we reduce the ImageCollection to a single Image by using the median method, which sets each pixel to the median value from all images covering that point.


In [13]:
def mask_clouds(image: ee.Image) -> ee.Image:
    '''Return the image with clouds masked'''
    
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = int(2 ** 10)
    cirrusBitMask = int(2 ** 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    image = image.updateMask(mask)
    
    return image

def filter_StudyArea(collection: ee.ImageCollection) -> ee.ImageCollection:
    '''Filter the collection to just those images overlapping the study area surrounding the sample coordinates'''
    # Define a rectangular area surrounding the sample coordinates with a +/- 0.5 degree buffer.
    StudyArea = ee.Geometry.Polygon([
        ee.Geometry.Point(WestBound, SouthBound),
        ee.Geometry.Point(WestBound, NorthBound),
        ee.Geometry.Point(EastBound, NorthBound),
        ee.Geometry.Point(EastBound, SouthBound)
    ])
    # Return a collection of just those images that are within the study area
    return collection.filterBounds(StudyArea)

Next we preprocess the collection. We filter the collection to include just those images that are in the study area and within a date range (adjust this as appropriate for your data). We apply the cloud masking function to each image, then divide each image by 10000 to correct the scale. This preprocessed collection is used as the input to the RasterDataset constructors.

In [14]:
sentinel_preprocessed = filter_StudyArea(
                            ee.ImageCollection("COPERNICUS/S2")
                        ).filterDate(
                            EarliestTime,
                            LatestTime
                        ).filter(
                            ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)
                        ).map(
                            lambda img: mask_clouds(img).divide(
                                ee.Image(10000)
                            )
                        ).median()

sentinel = [
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B1',
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B2',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B3',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B4',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B5'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B6'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B7'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B8'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B8A'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B9'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B10'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B11'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B12'
    ),
]

**Population density and structure**

Modeled population totals, as well as populations split by sex and age brackets, per hectare. This data is modeled for 2020 and described in detail [here](https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop_age_sex_cons_unadj). We convert population data from density per hectare to per square kilometer.


In [15]:
Population = [
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='population',
        name='total population per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_0',
        name='total male population age 0 to 1 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_1',
        name='total male population age 1 to 4 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_5',
        name='total male population age 5 to 9 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_10',
        name='total male population age 10 to 14 per hectare'
    ),
     RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_15',
        name='total male population age 15 to 19 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_20',
        name='total male population age 20 to 24 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_25',
        name='total male population age 25 to 29 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_30',
        name='total male population age 30 to 34 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_35',
        name='total male population age 35 to 39 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_40',
        name='total male population age 40 to 44 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_45',
        name='total male population age 45 to 49 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_50',
        name='total male population age 50 to 54 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_55',
        name='total male population age 55 to 59 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_60',
        name='total male population age 60 to 64 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_65',
        name='total male population age 65 to 65 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_70',
        name='total male population age 70 to 74 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_75',
        name='total male population age 75 to 79 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='M_80',
        name='total male population age 80 to 84 per hectare'
    ),
        RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_0',
        name='total female population age 0 to 1 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_1',
        name='total female population age 1 to 4 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_5',
        name='total female population age 5 to 9 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_10',
        name='total female population age 10 to 14 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_15',
        name='total female population age 15 to 19 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_20',
        name='total female population age 20 to 24 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_25',
        name='total female population age 25 to 29 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_30',
        name='total female population age 30 to 34 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_35',
        name='total female population age 35 to 39 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_40',
        name='total female population age 40 to 44 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_45',
        name='total female population age 45 to 49 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_50',
        name='total female population age 50 to 54 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_55',
        name='total female population age 55 to 59 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_60',
        name='total female population age 60 to 64 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_65',
        name='total female population age 65 to 65 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_70',
        name='total female population age 70 to 74 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_75',
        name='total female population age 75 to 79 per hectare'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj'),
        band='F_80',
        name='total female population age 80 to 84 per hectare'
    )
]

**Potential distribution of biomes**

Potential Natural Vegetation biomes global predictions of classes (based on predictions using the BIOMES 6000 dataset's 'current biomes' category.)

Potential Natural Vegetation (PNV) is the vegetation cover in equilibrium with climate that would exist at a given location non-impacted by human activities. PNV is useful for raising public awareness about land degradation and for estimating land potential. This dataset contains results of predictions of - (1) global distribution of biomes based on the BIOME 6000 data set (8057 modern pollen-based site reconstructions), - (2) distribution of forest tree species in Europe based on detailed occurrence records (1,546,435 ground observations), and - (3) global monthly Fraction of Absorbed Photosynthetically Active Radiation (FAPAR) values (30,301 randomly-sampled points).

This layer is described [here](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_PNV_PNV_BIOME-TYPE_BIOME00K_C_v01).


In [16]:
PotentialBiomes = [
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_BIOME-TYPE_BIOME00K_C/v01'),
        band='biome_type'
    )
]

**Accessibility to Healthcare**

This global accessibility map enumerates land-based travel time (in minutes) to the nearest hospital or clinic for all areas between 85 degrees north and 60 degrees south for a nominal year 2019. It also includes "walking-only" travel time, using non-motorized means of transportation only. This layer is described in detail [here](https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_accessibility_to_healthcare_2019).


In [17]:
HealthCareAccess = [
    RasterDataset(
        snippet=ee.Image('Oxford/MAP/accessibility_to_healthcare_2019'),
        band='accessibility',
        name='Travel time to nearest healthcare facility'
    ),
    RasterDataset(
        snippet=ee.Image('Oxford/MAP/accessibility_to_healthcare_2019'),
        band='accessibility_walking_only',
        name='Waling time to nearest healthcare facility'
    )
]

**Global Friction Surface**

This global friction surface enumerates land-based travel speed for all land pixels between 85 degrees north and 60 degrees south for a nominal year 2019. It also includes "walking-only" travel speed, using non-motorized means of transportation only. Layer details are described [here](https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2019).


In [18]:
GlobalFriction = [
    RasterDataset(
        snippet=ee.Image('Oxford/MAP/friction_surface_2019'),
        band='friction',
        name='Terrestrial travel speed'
    ),
    RasterDataset(
        snippet=ee.Image('Oxford/MAP/friction_surface_2019'),
        band='friction_walking_only',
        name='Walking speed'
    )
]

**Light pollution**

Monthly average radiance composite images using nighttime data from the Visible Infrared Imaging Radiometer Suite (VIIRS) Day/Night Band (DNB).

Layer details are [here](https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG).


In [19]:
LightPollution = [
    RasterDataset(
        snippet=ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').filterDate(
            EarliestTime, LatestTime
        ),
        band='avg_rad',
        name='Average Radiance'
    )
]

**Global SRTM CHILI (Continuous Heat-Insolation Load Index)**

CHILI is a surrogate for effects of insolation and topographic shading on evapotranspiration represented by calculating insolation at early afternoon, sun altitude equivalent to equinox. It is based on the 30m SRTM DEM (available in EE as USGS/SRTMGL1_003). Map layer described [here](https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_CHILI).

In [20]:
CHILI = [
    RasterDataset(
        snippet=ee.Image('CSP/ERGo/1_0/Global/SRTM_CHILI'),
        band='constant',
        name='Continuous Heat Insolation Load Index'
    )
]

**Global SRTM Landforms**

The SRTM Landform dataset provides landform classes created by combining the Continuous Heat-Insolation Load Index (SRTM CHILI) and the multi-scale Topographic Position Index (SRTM mTPI) datasets. It is based on the 30m SRTM DEM (available in EE as USGS/SRTMGL1_003). Map layer described [here](https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_landforms).

In [21]:
Landform = [
    RasterDataset(
        snippet=ee.Image('CSP/ERGo/1_0/Global/SRTM_landforms'),
        band='constant',
        name='Landform'
    )
]

**Potential Fraction of Absorbed Photosynthetically Active Radiation (FAPAR) Monthly**

Potential Natural Vegetation FAPAR predicted monthly median (based on PROB-V FAPAR 2014-2017). Map layer described [here](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_PNV_PNV_FAPAR_PROBA-V_D_v01).

In [22]:
FAPAR = [
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='jan',
        name='FAPAR January'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='feb',
        name='FAPAR February'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='mar',
        name='FAPAR March'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='apr',
        name='FAPAR April'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='may',
        name='FAPAR May'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='jun',
        name='FAPAR June'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='jul',
        name='FAPAR July'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='aug',
        name='FAPAR August'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='sep',
        name='FAPAR September'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='oct',
        name='FAPAR October'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='nov',
        name='FAPAR November'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='dec',
        name='FAPAR December'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='annual',
        name='FAPAR Annual'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/PNV/PNV_FAPAR_PROBA-V_D/v01'),
        band='annualdiff',
        name='FAPAR Annual difference'
    )
]

**Monthly precipitation**

Monthly precipitation in mm at 1 km resolution based on SM2RAIN-ASCAT 2007-2018, IMERG, CHELSA Climate, and WorldClim. Map layer is described in detail [here](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_CLM_CLM_PRECIPITATION_SM2RAIN_M_v01).

In [23]:
MonthlyPrecipitation = [
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='jan',
        name='January rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='feb',
        name='February rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='mar',
        name='March rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='apr',
        name='April rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='may',
        name='May rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='jun',
        name='June rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='jul',
        name='July rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='aug',
        name='August rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='sep',
        name='September rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='oct',
        name='October rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='nov',
        name='November rainfall'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01'),
        band='dec',
        name='December rainfall'
    )
]

**Ecoregions and realms**

The RESOLVE Ecoregions dataset, updated in 2017, offers a depiction of the 846 terrestrial ecoregions that represent our living planet. ECO_NAME is the name of the ecoregion. REALM is the name of the realm. Layer details are [here](https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017).

In [24]:
ecoregions = [
    VectorDataset(
        snippet=ee.FeatureCollection('RESOLVE/ECOREGIONS/2017'),
        property='ECO_NAME'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('RESOLVE/ECOREGIONS/2017'),
        property='REALM'
    )
]

**Watersheds**

HydroSHEDS is a mapping product that provides hydrographic information for regional and global-scale applications in a consistent format. It offers a suite of geo-referenced datasets (vector and raster) at various scales, including river networks, watershed boundaries, drainage directions, and flow accumulations. HydroSHEDS is based on elevation data obtained in 2000 by NASA's Shuttle Radar Topography Mission (SRTM).

HYBAS_ID = watershed ID number.

SUB_AREA = Area of basin, in square kilometers.

UP_AREA = Total upstream area, in square kilometers.

ENDO = Indicator for endorheic (inland) basins without surface flow connection to the ocean: 0 = not part of an endorheic basin; 1 = part of an endorheic basin; 2 = sink (i.e. most downstream polygon) of an endorheic basin.

COAST = Indicator for lumped coastal basins: 0 = no; 1 = yes. Coastal basins represent conglomerates of small coastal watersheds that drain into the ocean between larger river basins.

ORDER = Indicator of river order (classical ordering system): * Order 1 represents the main stem river from sink to source; * Order 2 represents all tributaries that flow into a 1st order river; * Order 3 represents all tributaries that flow into a 2nd order river; etc.; * Order 0 is used for conglomerates of small coastal watersheds.

Map details are [here](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_9).

In [25]:
watersheds = [
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='HYBAS_ID'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='SUB_AREA'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='UP_AREA'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='ENDO'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='COAST'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_9'),
        property='ORDER'
    )
]

**World Database on Protected Areas**

The World Database on Protected Areas (WDPA [link text](https://developers.google.com/earth-engine/datasets/catalog/WCMC_WDPA_current_polygons)) is the most up-to-date and complete source of information on protected areas, updated monthly with submissions from governments, non-governmental organizations, landowners, and communities. It is managed by the United Nations Environment Programme's World Conservation Monitoring Centre (UNEP-WCMC) with support from IUCN and its World Commission on Protected Areas (WCPA).

WDPAID = Unique identifier for a protected area (PA), assigned by UNEP-WCMC.

WDPA_PID = Unique identifier for parcels or zones within a PA, assigned by UNEP-WCMC.

DESIG_ENG = Designation of the PA in English. Allowed values for international-level designations: Ramsar Site, Wetland of International Importance; UNESCO-MAB Biosphere Reserve; or World Heritage Site. Allowed values for regional-level designations: Baltic Sea Protected Area (HELCOM), Specially Protected Area (Cartagena Convention), Marine Protected Area (CCAMLR), Marine Protected Area (OSPAR), Site of Community Importance (Habitats Directive), Special Protection Area (Birds Directive), or Specially Protected Areas of Mediterranean Importance (Barcelona Convention). No fixed values for PAs designated at a national level.

IUCN_CAT = IUCN management category, one of: Ia (strict nature reserve), Ib (wilderness area), II (national park), III (natural monument or feature), IV (habitat/species management area), V (protected landscape/seascape), VI (PA with sustainable use of natural resources), not applicable, not assigned, or not reported.

MARINE = This field describes whether a PA falls totally or partially within the marine environment, one of: 0 (100% terrestrial PA), 1 (coastal: marine and terrestrial PA), or 2 (100% marine PA).

REP_AREA = The total PA extent, including both marine (if applicable) and terrestrial areas, in square kilometers provided by data provider as specified in the legal text for the site.

STATUS_YR = Year of enactment of status in the STATUS field.

GOV_TYPE = Description of the decision-making structure of a PA. One of: federal or national ministry or agency, sub-national ministry or agency, government-delegated management, transboundary governance, collaborative governance, joint governance, individual landowners, non-profit organizations, for-profit organizations, indigenous peoples, local communities, or not reported.

In [26]:
ProtectedAreas = [
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='WDPAID'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='WDPA_PID'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='DESIG_ENG'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='IUCN_CAT'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='MARINE'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='REP_AREA'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='STATUS_YR'
    ),
    VectorDataset(
        snippet=ee.FeatureCollection('WCMC/WDPA/current/polygons'),
        property='GOV_TYPE'
    )
]

**Concatenate the lists of datasets**

In [27]:
# Define the list of datasets from which to retrieve data
datasets = bioclim + soil + terrain + HII + landsat + sentinel + Population + PotentialBiomes + HealthCareAccess + GlobalFriction + LightPollution + CHILI + Landform + FAPAR + MonthlyPrecipitation + ecoregions + watersheds + ProtectedAreas

In [28]:
# Here is an extra function for reprojecting an Image to a different scale. 
# Might be useful for something, not sure what effect this has exactly

def reproject(image: ee.Image, scale: int) -> ee.Image:
    '''
    Resample and reproject an image to a different resolution.
    
    param image: Image to reproject
    param scale: Desired pixel width in meters
    '''
    return image.resample('bilinear').reproject(  # Use bilinear interpolation method
        crs=image.projection().crs(),  # Keep the same map projection
        scale=scale  # Change the scale
    )




**Map the data**

Next we define a mapping class and some methods to help visualize the data. This will help with understanding what we're doing, and also make it easy to visually verify that the results make sense. The JavaScript version of the Earth Engine API provides a Map class that makes it easy to do this, but the Python API doesn't have that feature. I'm implementing it using Folium, a Python library for creating Leaflet Javascript maps, as recommended [here](https://developers.google.com/earth-engine/python_install-colab.html#interactive_map).


In [29]:
# These two methods are going to be added as custom folium map methods. 
# This will make it easy to display vector and raster Earth Engine objects to a map.
def add_raster_layer(self, ee_image_object, vis_params, name):
    """
    Display an EE image (raster) on a folium map
    
    param ee_image_object: ee.Image to add to the map
    param vis_params: map visualization parameters e.g. min, max, palette
    param name: name to give the layer in the layer control panel display
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

def add_vector_layer(self, coords, name):
    """
    Display an EE geometry (vector) on a folium map
    
    param ee_image_object: ee.Image to add to the map
    param vis_params: map visualization parameters e.g. min, max, palette
    param name: name to give the layer in the layer control panel display
    """
    # Reverse the order of the coordinates from (lng, lat) to (lat, lng)
    coords = np.array([np.flip(i) for i in coords])
    
    # Add the coordinates as a polygon layer in the map
    folium.vector_layers.Polygon(
        locations=coords,
        name=name,
        color='red',
        overlay=True,
        control=True,
        tooltip=name
    ).add_to(self)

Next we define the Map class, which makes use of the above methods to draw multiple datasets onto a folium map and display it.

In [30]:
class Map:
    
    def __init__(self):
        """Initialize a custom folium map"""
        # Add EE drawing methods to folium.
        folium.Map.add_raster_layer = add_raster_layer
        folium.Map.add_vector_layer = add_vector_layer
        
        # Centers the map on California. Adjust the center coordinates and zoom level for your data
        self.map = folium.Map(location=[35, -119], zoom_start=4, height=500)
        
    def add_polygon(self, coords: np.ndarray, name: str):
        """
        Add a polygon to the map
        
        param coords: array of (lng, lat) coordinate pairs outlining the polygon
        param name: name to give the polygon to display as a tooltip
        """
        self.map.add_vector_layer(coords, name)

    def add(self, dataset):
        """Add a dataset as a layer to an interactive map using folium"""

        # If the dataset is in vector format, we want to convert it to raster in order to 
        # display on the map more easily,
        if isinstance(dataset, VectorDataset):
            map_property = dataset.property
            
            # Non-numeric values can't be converted into raster format
            # We will have to arbitrarily assign numeric values to represent them
            if not isinstance(dataset.data.first().get(dataset.property), ee.Number):
                # Get an array of all existing values of the property
                values = ee.List(dataset.data.aggregate_array(dataset.property))
                # Use the index of each property in the array as an arbitrary numeric value
                # e.g. if your values were ['forest', 'desert', 'water'],
                # the new band would have values [0, 1, 2] respectively.
                dataset.data = dataset.data.map(
                    lambda feature: feature.set(
                        'as_number', values.indexOf(feature.get(dataset.property)))
                )
                # Do mapping based on the new numeric property
                map_property = 'as_number'
            
            # Convert to a raster image, turning values of the given property into pixel values
            image = dataset.data.reduceToImage(
                properties=[map_property], reducer=ee.Reducer.first())
            
        # If it's in raster format, we can use it as-is
        else:
            image = dataset.data
        
        self.map.add_raster_layer(
            image.updateMask(image.gt(0)), 
            dataset.map_params, 
            dataset.name)
        
    def get_coords(fc: ee.FeatureCollection) -> np.array:
        """
        Return a list of coordinate lists representing the geometries 
        of the features in the FeatureCollection.
        """
        coords = np.array(fc.iterate(
            lambda item, l: ee.List(l).add(item.geometry().coordinates()), 
            ee.List([])).getInfo())
        return coords
    
    def get_names(fc: ee.FeatureCollection) -> np.array:
        """
        Return a list of names extracted from the 'name' property of each 
        feature in the FeatureCollection.
        """
        names = np.array(fc.iterate(
            lambda item, l: ee.List(l).add(item.get('name')), 
            ee.List([])).getInfo())
        return names

    def display(self):
        """Display the map"""
        self.map.add_child(folium.LayerControl())  # Add a layer control panel to the map
        display(self.map)   # Display the map

**Run**

In [ ]:
# Define a map that will display all the data together, and add each sample area geometry to it
map = Map()

for polygon, name in zip(Map.get_coords(sample_areas), Map.get_names(sample_areas)):
    map.add_polygon(polygon, name=name)

# For each dataset,
for dataset in datasets:
    print(dataset.name)
    # Add a property to each sample area storing the value calculated from the dataset
    sample_areas = sample_areas.map(lambda feature: dataset.get_sample_area_data(feature))
    # Add the dataset as a layer to the map
    map.add(dataset)
    
map.display()  # Display the map

# Get the name of each dataset, which will be the column headers
dataset_names = [dataset.name for dataset in datasets]

# Export the sample area data to your Google Drive as a CSV
# Note: It may take a few minutes to show up.
task = ee.batch.Export.table.toDrive(
    collection=sample_areas, 
    description='eDNAMetadataOutput',  # The file will show up in your Drive with this name
    selectors=['name', 'sample_date', 'latitude', 'longitude', 'spatial_uncertainty'] + dataset_names  # Don't output geometry property
)
task.start()